In [1]:
import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

#Visualization Tools

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# AI Workflow Module
from model_classifier import ModelClassifier, CustomClassifier

# Models
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

# Use for saving model 
import joblib

# Environment Parameters

plt.rcParams['figure.figsize'] = [15, 8]
plt.rcParams['figure.dpi'] = 100 
plt.style.use('ggplot')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Data Preparation

In [2]:
train_df = pd.read_csv('../data/TCGA-SKCM_train_unresampled_v0.csv',index_col=0)
train_df.head()

sample_type  weight  height        bmi  age_at_index      A1BG         A2M      NAT1  NAT2  RP11-986E7.7  AADAC       AAMP   AANAT        AARS      ABAT      ABCA1      ABCA2      ABCA3     ABCB7      ABCF1    ABCA4       ABL1    AOC1        ABL2     ABO         ABR      ACAA1      ACACA     ACACB   ACADL      ACADM     ACADS    ACADSB     ACADVL      ACAT1     ACAT2   ASIC2     ASIC1      ACHE       ACLY       ACO1      ACR       ACO2      ACOX1       ACP1       ACP2       ACP5    ACPP   ACRV1    ACTA1      ACTA2        ACTG1      ACTG1.1    ACTC1      ACTG1.2      ACTG1.3     ACTG2       ACTN4     ACTL6A       ACTN1    ACTN2   ACTN3     ACVR1    ACVR1B    ACVR2A    ACVR2B    ACVRL1       ACY1     ACYP1    ACYP2       ADA     ADAM8     ADAM10        ADAR     ADARB1  ADARB2      ADCY1      ADCY2     ADCY3    ADCY5      ADCY6     ADCY7  ADCY8      ADCY9  ADCYAP1  ADCYAP1R1       ADD1     ADD2       ADD3      PLIN2  ADH1A    ADH1B   ADH1C    ADH4       ADH5    ADH6  ADH7  \
submitter_id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
TCGA-D3-A3MU            1   110.0   181.0  33.576509          53.0  118.1955  87810.7281   72.4179   0.0     2357.7364    0.0  2874.9505  0.0000  13314.6023   34.8239  1320.9300  4525.9201  3147.2101  291.6502  4302.7305   0.7915  4442.8176  5.9359   3444.7962  4.7487   3500.9893  1146.8144  1323.7040  528.2944  0.3957   476.8500  577.7602  221.6066  6764.9149   617.7285  212.9521  0.3957  187.9699  381.4800  5317.7681  1007.5188   3.5615  6456.3158   972.7701   965.1761  1400.4749  6477.6415  3.1658  0.0000  15.8290  4589.7467  154370.3997  113953.7238  22.9521  154370.3997  113953.7238  104.4717  24918.4488   875.3463  12840.5224  15.8290  0.7915  889.9881  701.6225  113.5734  265.5323  465.7697  1359.3194  167.3922  30.0752  418.2825  523.5457   962.4060  11755.4412  1276.6126   0.000  9111.9905    90.2256  810.9023   6.3316  2882.0736  278.5912    0.0  2224.3767   1.1872     0.7915  5513.2568   1.9786   824.2976   352.1963  0.000   0.3957  0.3957  0.0000  2471.7056  0.0000   0.0   
TCGA-EE-A180            1    88.0   173.0  29.607777          69.0  183.9909   8501.2061   74.2424   0.0        2.7273    0.0  1902.4242  0.3030   4952.1212    9.6970  1009.0909  1016.3636   471.5152  253.9394  2833.6364   0.3030  4578.1818  0.0000   6884.8485  0.9091   7060.0000   816.3636  2270.6061  516.0606  0.3030   643.9394  305.1515  140.3030  3604.2333   686.0606  338.3818  0.3030   24.8485    7.2727  6076.3636   283.9394   0.0000  3736.5727   890.6061   750.9091  1672.4242  1573.0303  1.8182  1.5152   6.9697   937.4667  146063.0273   97550.9091   0.6061  146063.0273   97550.9091   42.7273  23615.8091  1029.6970  10135.7576   4.5455  1.2121  819.3939  686.9697  213.0303  342.1212  173.3333   906.0273  126.0606  12.7273   85.4545   96.9697  1547.5758  26423.6364   555.7576   0.303    33.0303  2281.2121  659.6364   0.3030  1473.3333  119.6970    0.0  1051.5152   2.1212     1.5152  5715.1515   3.9394  1110.9091   209.3939  0.000  41.5152  0.0000  0.0000  2569.6970  1.2121   0.0   
TCGA-D3-A1

In [3]:
test_df = pd.read_csv('../data/TCGA-SKCM_test_unresampled_v0.csv',index_col=0)
test_df.head()

sample_type  weight  height        bmi  age_at_index      A1BG         A2M      NAT1    NAT2  RP11-986E7.7   AADAC       AAMP   AANAT       AARS      ABAT      ABCA1      ABCA2      ABCA3     ABCB7      ABCF1    ABCA4       ABL1     AOC1       ABL2      ABO        ABR      ACAA1      ACACA     ACACB    ACADL      ACADM      ACADS    ACADSB     ACADVL      ACAT1     ACAT2   ASIC2     ASIC1     ACHE       ACLY       ACO1     ACR        ACO2      ACOX1       ACP1       ACP2        ACP5      ACPP    ACRV1   ACTA1      ACTA2        ACTG1      ACTG1.1   ACTC1      ACTG1.2      ACTG1.3      ACTG2       ACTN4     ACTL6A       ACTN1     ACTN2   ACTN3      ACVR1    ACVR1B    ACVR2A     ACVR2B     ACVRL1       ACY1     ACYP1     ACYP2       ADA     ADAM8     ADAM10       ADAR     ADARB1  ADARB2      ADCY1      ADCY2      ADCY3     ADCY5      ADCY6     ADCY7    ADCY8      ADCY9  ADCYAP1  ADCYAP1R1       ADD1     ADD2       ADD3      PLIN2   ADH1A    ADH1B    ADH1C    ADH4  \
submitter_id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
TCGA-D3-A8GE            1    97.0   176.0  31.314566          26.0  195.8586  92516.2962   80.8254  0.4145     1469.3646  0.4145  2130.4750  0.4145  3919.4108  382.1591  1454.4098  2278.4477   257.3979  311.6960  1390.1971  22.3824  5254.0664   8.7043  1036.6377  44.3504  2301.2446   786.2862   807.0107  356.8753   0.0000   812.8135   294.2874  371.3824  7237.3811   647.4323  215.3355  3.3159   96.1615  12.8492  5511.4643  1148.5499  0.8290   2201.3817   926.6323   874.9869  1080.1591    169.9406  574.4822   0.4145  0.8290  7655.5262   95320.9339   57088.4406  7.4608   95320.9339   57088.4406   176.1580  35219.0648  1050.7304   3374.3574    0.8290  0.8290  2852.1009  912.2910  297.6033   175.7435  2118.0403   363.9672  101.9644  185.6912  141.3409  186.5202  1220.6710  7032.6400   962.4442  5.8029  1015.9133     2.0724  2597.2687  149.6306  2760.9132  717.0665   0.4145   954.1544   0.8290   291.3860  8301.3918  18.6520  6097.5521  2730.6555  1.6580   4.1449  75.4371  0.0000   
TCGA-D9-A4Z3            0    70.0   170.0  24.221453          73.0  502.4764   8233.4114   63.5752  0.0000      280.5043  0.0000  2157.2628  0.0000  4615.6490   37.3719   807.5775  1133.1859  2054.1678  670.5470  2369.8963   6.4434   829.0556   0.0000  2202.3669   0.0000  6223.5014   502.5881  1376.3182  272.7722   0.0000   915.8272   440.7311  523.6367  2871.1871  1366.0087  198.5567  1.7182  307.9963  14.6051  4527.1591  1231.1261  4.7252   6910.1140  1239.2878  1316.6090   828.1965   3646.9855    3.8661   8.5912  1.7182   632.1184   51654.4599  217038.5962  0.8591   51654.4599  217038.5962     9.8799  20685.4829  1101.3982  10836.1434  396.0566  2.1478   780.5150  435.5764  239.6959  1290.4057   114.6932   594.5145  133.1644   59.7092  269.7652   20.6190  2000.4725  4618.6559  2785.7128  0.0000  1839.8161     9.0208  2051.3929    1.7182  1423.5701  503.8768   0.0000  1093.2365   0.0000     1.2887  3646.9855  14.6051  1521.9399   628.8795  0.0000   0.0000   0.0000  0.0000   
TCGA-YD-A89C            1    59.0   164.0  21.

### Feature Selection

In [4]:
features = [
       'gender_male', 'race_not reported',
       'race_white',
       'treatments_pharmaceutical_treatment_or_therapy_not reported',
       'treatments_pharmaceutical_treatment_or_therapy_yes',
       'treatments_radiation_treatment_or_therapy_not reported',
       'treatments_radiation_treatment_or_therapy_yes',
       'age_at_index',
       'bmi',
       'C7', 'KRT17', 'CLEC2A', 'S100A7A', 'KRTDAP', 'WFDC5', 'KRT6B',
       'S100A7', 'KRT14', 'PVRL4'
]


model_classifier = ModelClassifier(
    train = train_df, 
    validation = test_df, 
    label = 'sample_type', 
    label_values = ['Primary Tumor', 'Metastatic'],
    features = features, 
    label_binarizer = False)

## Model Development

In [5]:
# Logistic Regression

lr_pipeline_params = [
    {
        'clf__estimator': [LogisticRegression(
                              solver='liblinear', 
                              max_iter=10000, 
                              tol=0.0001, 
                              fit_intercept=True)], 
        'clf__estimator__C': [1e-03, 1e-2, 1e-1, 1, 10, 100],
        'clf__estimator__penalty': ['l1', 'l2']    
    }
]

# Support Vector Machines - Linear Kernel

svm_linear_pipeline_params = [
    {   
        'clf__estimator': [SVC(probability=True, kernel = "linear")], 
        'clf__estimator__gamma': ['scale', 0.1,  0.01, 0.001, 0.00001, 1, 10],
        'clf__estimator__decision_function_shape': ["ovo", "ovr"]   
    }
]

# Support Vector Machines - Polynomial Kernel

svm_poly_pipeline_params = [
    {   
        'clf__estimator': [SVC(probability=True, kernel = "poly")], 
        'clf__estimator__gamma': ['scale', 0.1, 0.01, 0.001, 0.00001, 1, 10],
        'clf__estimator__decision_function_shape': ["ovo", "ovr"]   
    },
]

# Support Vector Machines - Radial Basis Kernel


svm_rbf_pipeline_params = [
    {   
        'clf__estimator': [SVC(probability=True, kernel = "rbf")], 
        'clf__estimator__gamma': ['scale', 0.1,  0.01, 0.001, 0.00001, 1, 10],
        'clf__estimator__decision_function_shape': ["ovo", "ovr"]   
    },
]

# Support Vector Machines - Sigmoid Kernel


svm_sigmoid_pipeline_params = [
    {   
        'clf__estimator': [SVC(probability=True, kernel = "sigmoid")], 
        'clf__estimator__gamma': ['scale', 0.1,  0.01, 0.001, 0.00001, 1, 10],
        'clf__estimator__decision_function_shape': ["ovo", "ovr"]   
    },
]

# Naive Bayes

gnb_pipeline_params = [
    {   
        'clf__estimator': [GaussianNB(priors = None)], 
    },
]

# Random Forest

rf_pipeline_params = [
    {   
        'clf__estimator': [RandomForestClassifier(random_state = 1, bootstrap = True)], 
        'clf__estimator__n_estimators': [30, 60, 70, 80, 90],
        'clf__estimator__max_features': [0.6, 0.65, 0.7, 0.75, 0.8],
        'clf__estimator__min_samples_leaf': [6, 8, 10, 12, 14],
        'clf__estimator__min_samples_split':  [2, 3, 5, 7]  
    },
]

# All Models in one pipeline without PCA

pipeline_params = [
    {
        'clf__estimator': [LogisticRegression(
                              solver='liblinear', 
                              max_iter=10000, 
                              tol=0.0001, 
                              fit_intercept=True)], 
        'clf__estimator__C': [1e-03, 1e-2, 1e-1, 1, 10, 100],
        'clf__estimator__penalty': ['l1', 'l2']    
    },
    {   
        'clf__estimator': [SVC(probability=True, kernel = "linear")], 
        'clf__estimator__gamma': ['scale', 0.1,  0.01, 0.001, 0.00001, 1, 10],
        'clf__estimator__decision_function_shape': ["ovo", "ovr"]   
    },
    {   
        'clf__estimator': [SVC(probability=True, kernel = "poly")], 
        'clf__estimator__gamma': ['scale', 0.1, 0.01, 0.001, 0.00001, 1, 10],
        'clf__estimator__decision_function_shape': ["ovo", "ovr"]   
    },
    {   
        'clf__estimator': [SVC(probability=True, kernel = "sigmoid")], 
        'clf__estimator__gamma': ['scale', 0.1,  0.01, 0.001, 0.00001, 1, 10],
        'clf__estimator__decision_function_shape': ["ovo", "ovr"]   
    },
    {   
        'clf__estimator': [SVC(probability=True, kernel = "rbf")], 
        'clf__estimator__gamma': ['scale', 0.1,  0.01, 0.001, 0.00001, 1, 10],
        'clf__estimator__decision_function_shape': ["ovo", "ovr"]   
    },
    {   
        'clf__estimator': [RandomForestClassifier(random_state = 1, bootstrap = True)], 
        'clf__estimator__n_estimators': [30, 60, 70, 80, 90],
        'clf__estimator__max_features': [0.6, 0.65, 0.7, 0.75, 0.8],
        'clf__estimator__min_samples_leaf': [6, 8, 10, 12, 14],
        'clf__estimator__min_samples_split':  [2, 3, 5, 7]   
    },
    {
        'clf__estimator': [GaussianNB(priors = None)]
    }

]

# All Models in one pipeline with PCA


pca_pipeline_params = [
    {
        'pca__n_components': [0.6, 0.7, 0.75, 0.8, 0.9],        
        'clf__estimator': [LogisticRegression(
                              solver='liblinear', 
                              max_iter=10000, 
                              tol=0.0001, 
                              fit_intercept=True)], 
        'clf__estimator__C': [1e-03, 1e-2, 1e-1, 1, 10, 100],
        'clf__estimator__penalty': ['l1', 'l2']    
    },
    {   
        'pca__n_components': [0.6, 0.7, 0.75, 0.8, 0.9],                
        'clf__estimator': [SVC(probability=True, kernel = "linear")], 
        'clf__estimator__gamma': [10.1, 0.01, 0.001, 0.00001, 10],
        'clf__estimator__decision_function_shape': ["ovo", "ovr"]   
    },
    {   
        'pca__n_components': [0.6, 0.7, 0.75, 0.8, 0.9],                
        'clf__estimator': [SVC(probability=True, kernel = "poly")], 
        'clf__estimator__gamma': [0.1, 0.01, 0.001, 0.00001, 10],
        'clf__estimator__decision_function_shape': ["ovo", "ovr"]   
    },
    { 
        'pca__n_components': [0.6, 0.7, 0.75, 0.8, 0.9],                
        'clf__estimator': [SVC(probability=True, kernel = "sigmoid")], 
        'clf__estimator__gamma': [1, 0.1, 0.01, 0.001, 0.00001, 10],
        'clf__estimator__decision_function_shape': ["ovo", "ovr"]   
    },
    {   
        'pca__n_components': [0.6, 0.7, 0.75, 0.8, 0.9],                
        'clf__estimator': [SVC(probability=True, kernel = "rbf")], 
        'clf__estimator__gamma': [1, 0.1, 0.01, 0.001, 0.00001, 10],
        'clf__estimator__decision_function_shape': ["ovo", "ovr"]   
    },
    { 
        'pca__n_components': [0.6, 0.7, 0.75, 0.8, 0.9],                
        'clf__estimator': [RandomForestClassifier(random_state=1, bootstrap = True)], 
        'clf__estimator__n_estimators': [30, 60, 70, 80, 90],
        'clf__estimator__max_features': [0.6, 0.65, 0.7, 0.75, 0.8],
        'clf__estimator__min_samples_leaf': [6, 8, 10, 12, 14],
        'clf__estimator__min_samples_split':  [2, 3, 5, 7]   
    },
    {
        'pca__n_components': [0.6, 0.7, 0.75, 0.8, 0.9],                
        'clf__estimator': [GaussianNB(priors = None)]
    }

]

### Model Performance [Without PCA]

#### Logistic Regression

In [6]:
grid_search_parameters = {
    "n_jobs": 4, 
    "cv": 2, 
    "return_train_score": False, 
    "verbose": 3, 
    "scoring": ["accuracy", "f1"],
    "refit": "f1"
}

lr_model = model_classifier.classifier(lr_pipeline_params, grid_search_parameters, pca = False)

model_classifier.classifier_metrics(lr_model, confusion_matrix = True)

Fitting 2 folds for each of 12 candidates, totalling 24 fits
Best parameter (CV score=0.889):
{'clf__estimator': LogisticRegression(C=10, max_iter=10000, penalty='l1', solver='liblinear'), 'clf__estimator__C': 10, 'clf__estimator__penalty': 'l1'}

Training Performance:
AUC score: 0.9288316243928765
LogLoss score: 0.28439984698593235
Accuracy: 0.8846153846153846
f1 macro: 0.8075252370755583
f1 micro: 0.8846153846153846
f1 weighted: 0.8714120996776243
Precision macro: 0.8709848053276813
Precision micro: 0.8709848053276813
Precision weighted: 0.8709848053276813
Recall macro: 0.7624123043712898
Recall micro: 0.8846153846153846
Recall weighted: 0.8846153846153846

Validation Performance:
AUC score: 0.9292002934702861
LogLoss score: 0.28226854961100073
Accuracy: 0.9186991869918699
f1 macro: 0.8744897959183673
f1 micro: 0.91869918699187
f1 weighted: 0.9138543222166915
Precision macro: 0.9101951626525026
Precision micro: 0.9101951626525026
Precision weighted: 0.9101951626525026
Recall macro: 0

### Getting weights of each Features

In [7]:
final_model = lr_model[1].estimator
coeff_df = pd.DataFrame(final_model.coef_[0])
coeff_df.index = model_classifier.train_features.columns
coeff_df.columns = ["weights"]
coeff_df.sort_values(by=['weights'], inplace=True, ascending=False)
coeff_df.to_csv('../data/Melanoma_Top10_Clinical_LR_weights.csv')
coeff_df

weights
KRTDAP                                              13.545353
C7                                                   4.859778
WFDC5                                                1.595499
treatments_radiation_treatment_or_therapy_yes        1.093486
race_not reported                                    0.780930
treatments_pharmaceutical_treatment_or_therapy_...   0.323006
race_white                                           0.301550
KRT14                                                0.096402
gender_male                                          0.072711
KRT17                                                0.000000
S100A7A                                              0.000000
KRT6B                                                0.000000
treatments_pharmaceutical_treatment_or_therapy_yes  -0.117368
bmi                                                 -0.207864
age_at_index                                        -0.307112
PVRL4                                               -0.383372
CLEC2A                                              -0.383717
treatments_radiation_treatment_or_therapy_not r...  -0.730709
S100A7                                             -27.908871

### Save the model for deployment

In [8]:
joblib.dump(lr_model, 'melanoma_combined_lr_classifier.pkl', compress=9)

['melanoma_combined_lr_classifier.pkl']